In [296]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from lib import utils
import os
from binance.spot import Spot
from dotenv import load_dotenv
load_dotenv()

True

In [269]:
GATEWAY_HOST = 'https://www.binance.com'
API_PATH = '/'.join([GATEWAY_HOST, 'bapi/futures/v1'])

endpoints = {
    "positions" : {
        "path": 'public/future/copy-trade/lead-data/positions?portfolioId=%s', 
        "type": "simple",
        "params": {}
        },
    "performance" : {
        "path": 'public/future/copy-trade/lead-portfolio/performance?portfolioId=%s&timeRange=%s',
        "type": "simple",
        "params": {"timeRange": "90D"}
        },
    "detail" : {
        "path": 'friendly/future/copy-trade/lead-portfolio/detail?portfolioId=%s', 
        "type": "simple",
        "params": {}
    },
    "chart" : {
        "path": 'public/future/copy-trade/lead-portfolio/chart-data?portfolioId=%s&timeRange=%s&dataType=%s', 
        "type": "simple",
        "params": {"timeRange": "90D", "dataType": "ROI"}
    },
    "position_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/position-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    },
    "transfer_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/transfer-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    }
}


async def fetch_data(leaderId, endpointType, params={}):
    endpoint = endpoints[endpointType]
    # Filter out the empty params
    filtered_params = {}
    for default_key, default_value in endpoint['params'].items():
        if default_key in params.keys() and params[default_key] is not None:
            filtered_params[default_key] = params[default_key]
        else:
            filtered_params[default_key] = default_value

    if endpoint["type"] == 'simple':
        # Interpolate strings
        path = endpoint["path"] % (leaderId, *filtered_params.values())
        url = '/'.join([API_PATH, path])

        response = requests.get(url)

    if endpoint["type"] == 'paginated':
        url = '/'.join([API_PATH, endpoint["path"]])

        response = requests.post(
            url,
            json={"portfolioId": leaderId} | filtered_params,
            )

    return response.json()


async def fetch_pages(leaderId, endpointType, params={}, page_number=None, result=None, latest_item=None, reference=None, progress_bar=None):
    if page_number is None:
        page_number = 1
    if result is None:
        result = []

    response = await fetch_data(leaderId, endpointType, {"pageNumber": page_number} | params)

    if response["success"]:
        response_data = response["data"]
        response_list = response_data["list"]

        if latest_item and reference:
            response_list = sorted(response_list, key=lambda x: x[reference], reverse=True)
            item_index = 0

            for item in response_list:
                if item[reference] > latest_item[reference]:
                    result.append(item)
                    item_index += 1

                    if item_index == 10:
                        next_page = page_number + 1
                        return await fetch_pages(leaderId, endpointType, params, next_page, result, latest_item, reference)
                else:
                    # print({ "success": True, "reason": "partial", "message": f"Fetched pages {endpointType} - finished by update", "data": result })
                    return result
        else:
            result += response_list
            total_n_results = response_data["total"]
            pages_length = total_n_results // 10

            if progress_bar is None:
                progress_bar = tqdm(total=total_n_results)

            progress_bar.update(len(response_list))
            # If remainder, add another page
            if total_n_results % 10:
                pages_length += 1

            if page_number <= pages_length:
                next_page = page_number + 1

                return await fetch_pages(leaderId, endpointType, params, next_page, result, progress_bar=progress_bar)
            else:
                return result
    
    else:
        return { "success": False, "message": f"Could not fetch page {page_number}/{pages_length} of {endpointType}" }
    

In [281]:
# leader_ids = ["3846188874749232129", "3842534998056366337"]
leader_ids = ["3846188874749232129"]
leader = {}

for leader_id in leader_ids:
    details = await fetch_data(leader_id, 'detail')
    leader_details = details["data"]
    
    positions = await fetch_data(leader_id, 'positions')
    leader_positions = pd.DataFrame(positions["data"])
    leader_positions["LEADER_ID"] = leader_id
    
    position_history = await fetch_pages(leader_id, 'position_history')
    leader_position_history = pd.DataFrame(position_history)
    leader_position_history["LEADER_ID"] = leader_id

    transfer_history = await fetch_pages(leader_id, 'transfer_history')
    leader_transfer_history = pd.DataFrame(transfer_history)
    leader_transfer_history["LEADER_ID"] = leader_id

    performance = await fetch_data(leader_id, 'performance')
    leader_performance = performance["data"]
    
leader_positions.head()

100%|██████████| 11/11 [00:00<00:00, 17.10it/s]


,id,symbol,collateral,positionAmount,entryPrice,unrealizedProfit,cumRealized,askNotional,bidNotional,notionalValue,markPrice,leverage,isolated,isolatedWallet,adl,positionSide,breakEvenPrice,LEADER_ID
0,0_SUSHIUSDT_BOTH,SUSHIUSDT,USDT,0,0.0,0.00000000,0,0,0,0,1.52450000,1,True,0,0,BOTH,0.0,3846188874749232129
1,0_SUSHIUSDT_LONG,SUSHIUSDT,USDT,0,0.0,0.00000000,0,0,0,0,1.52450000,1,True,0,0,LONG,0.0,3846188874749232129
2,0_SUSHIUSDT_SHORT,SUSHIUSDT,USDT,0,0.0,0.00000000,-422.08230134,0,0,0,1.52450000,1,True,0,0,SHORT,0.0,3846188874749232129
3,0_BNBUSDT_BOTH,BNBUSDT,USDT,0.00,0.0,0.00000000,0,0,0,0,608.36000000,1,True,0,0,BOTH,0.0,3846188874749232129
4,0_BNBUSDT_LONG,BNBUSDT,USDT,0.00,0.0,0.00000000,0,0,0,0,608.36000000,1,True,0,0,LONG,0.0,3846188874749232129


In [282]:
leader_transfer_history["INCOMING"] = leader_transfer_history["to"] == 'Lead Trading Account'
leader_transfer_history["RELATIVE_VALUE"] = leader_transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)

leader_transfer_history.head()

,time,coin,amount,from,to,transType,LEADER_ID,INCOMING,RELATIVE_VALUE
0,1711015734693,USDT,62523.139709,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,62523.139709
1,1710910426929,USDT,2109.074057,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,2109.074057
2,1710631903556,USDT,48177.912951,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48177.912951
3,1710631606472,USDT,48177.912951,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,48177.912951
4,1710631512228,USDT,48175.731055,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48175.731055


In [283]:
leader_position_history

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime,LEADER_ID
0,28831399,BCHUSDT,UM,1712752315689,1.712785e+12,603.280501,618.462375,-945.041280,62.248,62.248,Isolated,Short,All Closed,1712784529622,3846188874749232129
1,28675086,BCHUSDT,UM,1712711525798,1.712729e+12,666.038565,626.704920,2201.700750,55.975,55.975,Isolated,Short,All Closed,1712728550720,3846188874749232129
2,28368782,ETHUSDT,UM,1712627977755,NaN,3603.292522,3490.719604,-6972.078457,45.070,44.950,Isolated,Long,Partially Closed,1712784543076,3846188874749232129
3,27731385,BCHUSDT,UM,1712373128871,1.712461e+12,702.900561,684.506554,905.592130,49.233,49.233,Isolated,Short,All Closed,1712461052390,3846188874749232129
4,27604253,ETHUSDT,UM,1712319921263,1.712628e+12,3296.424227,3565.913762,16574.414869,41.217,61.503,Isolated,Long,All Closed,1712665128410,3846188874749232129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10433832,ETHUSDT,UM,1707831600369,1.707834e+12,2641.025157,2610.031900,-2356.820250,76.043,76.043,Isolated,Long,All Closed,1707834104910,3846188874749232129
96,10225346,ETHUSDT,UM,1707732084449,1.707831e+12,2483.687340,2576.053026,20238.614850,146.525,219.114,Isolated,Long,All Closed,1707831450716,3846188874749232129
97,9987905,ETHUSDT,UM,1707565217287,1.707732e+12,2479.567046,2473.923819,-832.652429,147.549,147.549,Isolated,Long,All Closed,1707731638248,3846188874749232129
98,9450605,ETHUSDT,UM,1707209329727,1.707565e+12,2334.502083,2476.586570,19930.475049,140.272,140.272,Cross,Long,All Closed,1707565060277,3846188874749232129


In [284]:
sum_columns = [
    "positionAmount",
    "unrealizedProfit",
    "cumRealized",
    "notionalValue",
	"markPrice",
    "ABSOLUTE_LEVERED_VALUE",
    "ABSOLUTE_UNLEVERED_VALUE"
]

average_columns = [
    "entryPrice",
	"markPrice",
	"leverage",
	"breakEvenPrice",
]

drop_columns = [
    "id",
    "collateral",
    "isolated",
    "isolatedWallet",
    "adl",
    "askNotional",
    "bidNotional"
]

filtered_leader_positions = leader_positions.apply(lambda column: column.astype(float) if column.name in sum_columns + average_columns else column)
filtered_leader_positions = filtered_leader_positions.loc[(filtered_leader_positions["positionAmount"] != 0) | (filtered_leader_positions["collateral"] != "USDT")]
filtered_leader_positions = filtered_leader_positions.drop(columns=drop_columns)
filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] = abs(filtered_leader_positions["notionalValue"])
filtered_leader_positions["ABSOLUTE_UNLEVERED_VALUE"] = filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] / filtered_leader_positions["leverage"]
filtered_leader_positions

,symbol,positionAmount,entryPrice,unrealizedProfit,cumRealized,notionalValue,markPrice,leverage,positionSide,breakEvenPrice,LEADER_ID,ABSOLUTE_LEVERED_VALUE,ABSOLUTE_UNLEVERED_VALUE
13,ETHUSDT,43.252,3559.088189,-736.746669,95554.626641,153200.935664,3542.054371,2.0,LONG,3725.772762,3846188874749232129,153200.935664,76600.467832


In [285]:

def aggregate_positions(group: pd.DataFrame) -> pd.Series:
    result = {}
    
    for column in sum_columns: result[column] = np.sum(group[column])
    for column in average_columns: result[column] = np.average(group[column], weights=group["positionAmount"])

    if len(group) > 1: result["positionSide"] = "BOTH"
    else: result["positionSide"] = group["positionSide"].values[0]

    # result["LEADER_IDS"] = list(set(group["LEADER_ID"]))
    return pd.Series(result)

grouped_leader_positions = filtered_leader_positions.groupby("symbol").apply(aggregate_positions, include_groups=False)
grouped_leader_positions = grouped_leader_positions.rename(columns={key: key + "_SUM" for key in sum_columns} | {key: key + "_AVERAGE" for key in average_columns})
grouped_leader_positions["LEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"] / np.sum(grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"])
grouped_leader_positions["UNLEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"] / np.sum(grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"])

grouped_leader_positions

,positionAmount_SUM,unrealizedProfit_SUM,cumRealized_SUM,notionalValue_SUM,markPrice_AVERAGE,ABSOLUTE_LEVERED_VALUE_SUM,ABSOLUTE_UNLEVERED_VALUE_SUM,entryPrice_AVERAGE,leverage_AVERAGE,breakEvenPrice_AVERAGE,positionSide,LEVERED_POSITION_SHARE,UNLEVERED_POSITION_SHARE
symbol,,,,,,,,,,,,,
ETHUSDT,43.252,-736.746669,95554.626641,153200.935664,3542.054371,153200.935664,76600.467832,3559.088189,2.0,3725.772762,LONG,1.0,1.0


In [286]:
leader_details

{'userId': 178325873,
 'leadOwner': False,
 'hasCopy': False,
 'leadPortfolioId': '3846188874749232129',
 'nickname': '午后阳光',
 'nicknameTranslate': None,
 'avatarUrl': 'https://public-1306379396.file.myqcloud.com/image/common_notification/20211230/ca5ef4d8-dc73-43ba-9b42-f0ed45929ac5.png',
 'status': 'ACTIVE',
 'description': '只做2倍合约中长线波段，稳定复利，短线勿扰',
 'descTranslate': None,
 'favoriteCount': 3537,
 'currentCopyCount': 147,
 'maxCopyCount': 500,
 'totalCopyCount': 1002,
 'marginBalance': '76392.02987244',
 'initInvestAsset': 'USDT',
 'futuresType': 'UM',
 'aumAmount': '270114.72086068',
 'copierPnl': '25422.50043790',
 'copierPnlAsset': 'USDT',
 'profitSharingRate': '10.00',
 'unrealizedProfitShareAmount': '0',
 'startTime': 1707209121766,
 'endTime': None,
 'closedTime': None,
 'tag': [],
 'positionShow': True,
 'mockCopyCount': 4517,
 'sharpRatio': '6.06915003',
 'hasMock': False,
 'lockPeriod': None,
 'copierLockPeriodTime': None,
 'badgeName': None,
 'badgeModifyTime': None,
 'badge

In [289]:
transfer_balance = np.sum(leader_transfer_history["RELATIVE_VALUE"])
historic_PNL = np.sum(leader_position_history["closingPnl"])
total_balance = transfer_balance + historic_PNL
updated = utils.current_time()

leader = {
    "updated": updated,
    "updated_date": utils.current_readable_time(),
    "details": {
        "updated": updated,
        "data": leader_details
    },
    # "positions": filtered_leader_positions.to_dict(),
    # "mix": grouped_leader_positions.to_dict(),
    "account": {
        "updated": updated,
        "transfer_balance": transfer_balance,
        "historic_PNL": historic_PNL,
        "total_balance": total_balance,
        "levered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"]) / total_balance,
        "unlevered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"]) / total_balance
    },
    "performance": {
        "updated": updated,
        "data": leader_performance

    }
}

leader

{'updated': 1712790965901,
 'updated_date': '2024-04-11 01:16:05',
 'details': {'updated': 1712790965901,
  'data': {'userId': 178325873,
   'leadOwner': False,
   'hasCopy': False,
   'leadPortfolioId': '3846188874749232129',
   'nickname': '午后阳光',
   'nicknameTranslate': None,
   'avatarUrl': 'https://public-1306379396.file.myqcloud.com/image/common_notification/20211230/ca5ef4d8-dc73-43ba-9b42-f0ed45929ac5.png',
   'status': 'ACTIVE',
   'description': '只做2倍合约中长线波段，稳定复利，短线勿扰',
   'descTranslate': None,
   'favoriteCount': 3537,
   'currentCopyCount': 147,
   'maxCopyCount': 500,
   'totalCopyCount': 1002,
   'marginBalance': '76392.02987244',
   'initInvestAsset': 'USDT',
   'futuresType': 'UM',
   'aumAmount': '270114.72086068',
   'copierPnl': '25422.50043790',
   'copierPnlAsset': 'USDT',
   'profitSharingRate': '10.00',
   'unrealizedProfitShareAmount': '0',
   'startTime': 1707209121766,
   'endTime': None,
   'closedTime': None,
   'tag': [],
   'positionShow': True,
   'mockC

In [329]:
user = {
    "updated": updated,
    "updated_date": utils.current_readable_time(),
    "auth": {
        "updated": updated,
        "username": "root",
        "email": "root@example.com",
        "password_hash": '',
        "binance_api_key": '',
        "binance_secret_hash": '',
    },
    "details": {
        "updated": updated,
        "active": True,
        "chat_id": 1031182213
    },
    "account": {
        "updated": updated,
        "leverage": 5,
        "value_USDT": 0,
        "value_BTC": 0,
        "levered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"]) / total_balance,
        "unlevered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"]) / total_balance,
        "collateral_margin_level": 0,
        "collateral_value_USDT": 0,
        "leaders": {},
        "positions":{},
        "mix": {}
    }
}

BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_SECRET_KEY = os.environ.get("BINANCE_SECRET_KEY")

client = Spot(api_key=BINANCE_API_KEY, api_secret=BINANCE_SECRET_KEY)

async def database_update(obj: object, update: object, collection: str) -> bool:
    current_time = utils.current_time()

    obj["updated"] = current_time
    obj["updated_date"] = utils.current_readable_time()

    for update_key in update.keys():
        update[update_key]["updated"] = current_time

    obj.update(update)

    # self.app.db[collection].update_one({"_id": obj["_id"]}, {"$set": update})

    return True

def user_account_update(): #self, user
    weigth = 10

    margin_account_data = client.margin_account()

    positions = []
    for asset in margin_account_data["userAssets"]:
        amount = float(asset["netAsset"])
        if amount != 0: positions.append(asset)

    assetBTC = float(margin_account_data["totalNetAssetOfBtc"])
    valueUSDT = float(client.ticker_price("BTCUSDT")["price"]) * assetBTC

    user_account_update = {
        "account": {
            "leverage": 5,
            "value_BTC": assetBTC,
            "value_USDT": valueUSDT,
            "levered_ratio": None,
            "unlevered_ratio": None,
            "collateral_margin_level": float(margin_account_data["totalCollateralValueInUSDT"]),
            "collateral_value_USDT": float(margin_account_data["collateralMarginLevel"]),
            "positions": positions,
        },
    }

    return user_account_update


In [330]:
user_account = user_account_update()
user_account_update_success = await database_update(user, user_account, 'users')
pd.DataFrame(user["account"]["positions"])

,asset,free,locked,borrowed,interest,netAsset
0,MATIC,0,0,0.0118,0,-0.0118
1,APT,1.18881,0,0,0,1.18881
2,SHIB,0,0,74687.7,0,-74687.7
3,WLD,1.8981,0,0,0,1.8981
4,BCH,0.000984,0,0.001447,0,-0.000463
5,CFX,47.952,0,0,0,47.952
6,ACE,1.1988,0,0,0,1.1988
7,TIA,1.0989,0,0,0,1.0989
8,BLUR,33.1668,0,0,0,33.1668
9,DOGE,6.293,0,0,0,6.293
